In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:100%;}</style>"))

import bokeh
import datetime as dt
import pandas as pd
from pandasql import sqldf
import plotly
import quandl

from dontAddThis import *  #def a func returning your own api key.

In [2]:
quandl.ApiConfig.api_key = quandlApi()

In [3]:
dow30Plus = ['AXP', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'XOM', 'GE', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 
             'TRV', 'UNH', 'UTX', 'VZ', 'V', 'WMT', 'DIS', 'DWDP', 'PEP', 'COF', 'WFC', 'AMZN']
test3Stocks = ['AXP', 'AAPL', 'BA']

In [4]:
#########################  Wiki Prices Data  #########################
wikiPrices = quandl.get_table('WIKI/PRICES', ticker = dow30Plus, date = { 'gte': '2011-01-01', 'lte': dt.datetime.today().strftime('%Y-%m-%d') }, paginate=True)

In [5]:
wikiPrices = wikiPrices[['ticker', 'date', 'adj_close', 'adj_volume']]
wikiPrices = wikiPrices.sort_values(['ticker','date'])
wikiPrices.tail()
#wikiPrices.to_csv('wikiPrices_dow30Plus.csv')

,ticker,date,adj_close,adj_volume
None,,,,
59165,XOM,2018-02-12,76.42,19095387.0
59166,XOM,2018-02-13,76.30,11846694.0
59167,XOM,2018-02-14,76.46,16741063.0
59168,XOM,2018-02-15,76.21,14030212.0
59169,XOM,2018-02-16,76.54,13674350.0


In [6]:
#########################  Zacks FR Data  #########################
zacksFR = quandl.get_table('ZACKS/FR', ticker = dow30Plus)

In [7]:
zacksFR = zacksFR[zacksFR.per_type=='Q'][['ticker', 'per_end_date', 'profit_margin', 'free_cash_flow_per_share']]
zacksFR = zacksFR.sort_values(['ticker', 'per_end_date'])
zacksFR.tail(2)
#zacksFR.groupby('ticker').count()

,ticker,per_end_date,profit_margin,free_cash_flow_per_share
None,,,,
1014,XOM,2017-09-30,6.0002,2.752
1015,XOM,2017-12-31,NaN,NaN


In [8]:
#########################  Zacks FC Data  #########################
zacksFC = quandl.get_table('ZACKS/FC', ticker = dow30Plus)

In [9]:
zacksFC = zacksFC[zacksFC.per_type=='Q'][['ticker', 'per_end_date', 'filing_date', 'eps_diluted_net', 'end_cash', 'incr_decr_cash']]
zacksFC = zacksFC.sort_values(['ticker', 'per_end_date'])
zacksFC.tail(2)
#zacksFC.groupby('ticker').count()

,ticker,per_end_date,filing_date,eps_diluted_net,end_cash,incr_decr_cash
None,,,,,,
1014,XOM,2017-09-30,2017-11-01,0.93,4266.0,609.0
1015,XOM,2017-12-31,NaT,1.97,NaN,NaN


In [10]:
#left join zacksFR to zacksFC; TODO:  use filing date to join on price data (and ffill until next Q10), as that's when market learns of results
pysqldf = lambda q: sqldf(q, globals())
bothZacks = pysqldf("""SELECT a.ticker, a.per_end_date, a.filing_date, a.eps_diluted_net, a.end_cash, a.incr_decr_cash, b.free_cash_flow_per_share, b.profit_margin
            FROM zacksFC AS a 
            LEFT JOIN zacksFR AS b ON a.ticker = b.ticker AND a.per_end_date = b.per_end_date;""")

bothZacks_and_price = pysqldf("""SELECT a.*, b.filing_date, b.eps_diluted_net, b.end_cash, b.incr_decr_cash, b.free_cash_flow_per_share, b.profit_margin
                    FROM wikiPrices AS a 
                    LEFT JOIN bothZacks AS b ON a.ticker = b.ticker AND a.date = b.filing_date     
                    """)

In [11]:
# probably best to group and apply ffill here than add more computation to later flask route
bothZacks_and_price_ffilled = bothZacks_and_price.groupby('ticker').ffill()

bothZacks_and_price_ffilled['date'] = pd.to_datetime(bothZacks_and_price_ffilled['date'])
bothZacks_and_price_ffilled = bothZacks_and_price_ffilled[bothZacks_and_price_ffilled['date'] > '2011-12-31']

bothZacks_and_price_ffilled.head()
bothZacks_and_price_ffilled.to_csv('bothZacks_and_price_ffilled.csv')

In [12]:
#TODO:  pull gdp annualized quarterly gdp (perhaps unemployment, etc.) data and join before defining Bokeh functions.  
#If compute on interactivity is trivial: join on fly, else: accept repeat data across all tickers
#Must calculate tooltip (or mtext in a <span>?) values such as highest PE, return by ticker, comparable return on SP500, etc. per selected period on fly!

In [13]:
#print(bothZacks_and_price_ffilled[bothZacks_and_price_ffilled.ticker == 'AAPL'].shape[0])
msftDays = bothZacks_and_price_ffilled[bothZacks_and_price_ffilled.ticker == 'MSFT']['date']
aaplDays = bothZacks_and_price_ffilled[bothZacks_and_price_ffilled.ticker == 'AAPL']['date']
set(msftDays) - set(aaplDays) # what\'s up with 2017-08-07??  Maybe (1) just linearly interpolate and notate in tooltip, if this is unlikely and rare...? (2) remove non set records.

{Timestamp('2017-08-07 00:00:00')}

In [14]:
#bothZacks_and_price_ffilled[bothZacks_and_price_ffilled['date']=='2017-08-07'].sort_values('ticker')
#bothZacks_and_price_ffilled.groupby('ticker').size().sort_values()
bothZacks_and_price_ffilled.dtypes

None                                 int64
ticker                              object
date                        datetime64[ns]
adj_close                          float64
adj_volume                         float64
filing_date                         object
eps_diluted_net                    float64
end_cash                           float64
incr_decr_cash                     float64
free_cash_flow_per_share           float64
profit_margin                      float64
dtype: object

In [15]:
from bokeh.io import notebook, curdoc
from bokeh.models import HoverTool, Select
from bokeh.plotting import figure, output_file, show, ColumnDataSource

# use try except for possible instance of adding new ticker that IPO'd later than all others.

myTicker1 = 'AAPL'
myTicker1_date = bothZacks_and_price_ffilled[bothZacks_and_price_ffilled.ticker == myTicker1]['date']
myTicker1_adj_close = bothZacks_and_price_ffilled[bothZacks_and_price_ffilled.ticker == myTicker1]['adj_close']
myTicker1_symbol = [myTicker1] * bothZacks_and_price_ffilled[bothZacks_and_price_ffilled.ticker == myTicker1]['ticker'].shape[0]

myTicker2 = 'AMZN'
myTicker2_date = bothZacks_and_price_ffilled[bothZacks_and_price_ffilled.ticker == myTicker2]['date']
myTicker2_adj_close = bothZacks_and_price_ffilled[bothZacks_and_price_ffilled.ticker == myTicker2]['adj_close']
myTicker2_symbol = [myTicker2] * bothZacks_and_price_ffilled[bothZacks_and_price_ffilled.ticker == myTicker2]['ticker'].shape[0]

source = ColumnDataSource(data=dict(myTicker1_date = myTicker1_date,                                    
                                    myTicker2_date = myTicker2_date,                                    
                                    myTicker1_adj_close=myTicker1_adj_close, 
                                    myTicker2_adj_close=myTicker2_adj_close, 
                                    myTicker1_symbol=myTicker1_symbol,
                                    myTicker2_symbol=myTicker2_symbol))

#TODO:  how to apply tooltip by glyph?
hover = HoverTool(tooltips = [('Trading Day', '@{myTicker1_date}{%F}'), (myTicker1, '@{myTicker1_adj_close}{($ 0.000 a)}'), (myTicker2, '@{myTicker2_adj_close}{($ 0.000 a)}')],
                 formatters={'myTicker1_date':'datetime'}
                 )

plot = figure(x_axis_label='Trading Date', y_axis_label='Adjusted Close', sizing_mode='stretch_both', x_axis_type='datetime', tools=[hover, 'box_select'])

plot.line('myTicker1_date', 'myTicker1_adj_close', source=source, line_width=2)
plot.line('myTicker2_date', 'myTicker2_adj_close', source=source, line_width=2)

output_file('aapl_amzn_static1.html')

show(plot)